# Librerías

In [ ]:
import requests as rq
import json
from types import SimpleNamespace
import pandas as pd
import time
from google.colab import drive

# Carga de las coordenadas de las viviendas

In [ ]:
# Vinculamos nuestro Drive personal
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargamos los datos
path = "/content/drive/MyDrive/arnau.csv"
dataset = pd.read_csv(path)
dataset = dataset[["latitude","longitude"]]
dataset.head()

,latitude,longitude
0,40.385300,-3.726606
1,40.378166,-3.730286
2,40.379831,-3.731038
3,40.377666,-3.733393
4,40.386182,-3.732559


In [ ]:
# Número de registros que contiene el dataset
print("El dataset contiene un total de {} documentos.\n".format(len(dataset)))
# Número de duplicados
print("Existen {} registros duplicados.\n".format(dataset.duplicated().sum()))
# Valores nulos
print("Vemos si el dataset contiene valores nulos:\n", dataset.isnull().sum())

El dataset contiene un total de 580 documentos.

Existen 0 registros duplicados.

Vemos si el dataset contiene valores nulos:
 latitude     0
longitude    0
dtype: int64


# Introducir la KEY

Según lo visto durante las pruebas de la API, para cada conjunto de parámetros de entrada se obtiene información sobre **60 lugares**, como máximo. Dado que cada petición devuelve 20 lugares, se pueden realizar hasta un máximo de 3 peticiones para obtener el máximo de información para cada conjunto de parámetros.

Se introduce la KEY y se ejecutan las celdas siguientes, automáticamente se lanzan hasta 3 peticiones por cada par de coordenadas y tipo de establecimiento. Interesan 3 tipos: "restaurant", "bar", "night_club".

El radio de búsqueda se fija en 750m.

En resumen, se lanzan hasta un máximo de 9 peticiones por cada par de coordenadas, correspondiente a una vivienda.

In [ ]:
KEY = ""

# EJECUTAR TODO

## Funciones

In [ ]:
def nearby_search(key, lat, lon, radius, find_type, next_page_token=None):
  response = rq.get(url="https://maps.googleapis.com/maps/api/place/nearbysearch/json", 
                  params={
                      "location":f"{lat}, {lon}",
                      "type": find_type,
                      "radius": radius,
                      "key": key,
                      "pagetoken": next_page_token
                      })
  return json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))

def peticion_individual(KEY, latitud, longitud, radio, tipo):
  #results = []
  response = nearby_search(KEY, latitud, longitud, radio, tipo)
  for element in response.results:
      results.append({
          "place_id": element.place_id,
          "name": element.name,
          "business_status": element.business_status if "business_status" in list(element.__dict__) else "NOT DEFINED",
          "rating": element.rating if "rating" in list(element.__dict__) else -1,
          "user_ratings_total": element.user_ratings_total if "user_ratings_total" in list(element.__dict__) else -1,
          "price_level": element.price_level if "price_level" in list(element.__dict__) else -1,
          "scope": element.scope,
          "types": element.types,
          "find_type": tipo,
          "lat": element.geometry.location.lat,
          "lon": element.geometry.location.lng
      })
  
  while "next_page_token" in list(response.__dict__):
    time.sleep(2)
    response = nearby_search(KEY, latitud, longitud, radio, tipo, response.next_page_token)
    for element in response.results:
      results.append({
          "place_id": element.place_id,
          "name": element.name,
          "business_status": element.business_status if "business_status" in list(element.__dict__) else "NOT DEFINED",
          "rating": element.rating if "rating" in list(element.__dict__) else -1,
          "user_ratings_total": element.user_ratings_total if "user_ratings_total" in list(element.__dict__) else -1,
          "price_level": element.price_level if "price_level" in list(element.__dict__) else -1,
          "scope": element.scope,
          "types": element.types,
          "find_type": tipo,
          "lat": element.geometry.location.lat,
          "lon": element.geometry.location.lng
     })
  return results

## Petición

In [ ]:
# Parámetros de entrada
tipos = ["restaurant","bar","night_club"]
radio = 750

# Peticiones
results = []

for tipo in tipos:
  for lat, lon in zip(dataset.latitude,dataset.longitude):
    peticion_individual(KEY, lat, lon, radio, tipo)

# Guardar el .csv de Salida

In [ ]:
final_data = pd.DataFrame(results)
# Número de registros que contiene el dataset
print("El dataset contiene un total de {} registros.\n".format(len(final_data)))
# Número de duplicados
print("Se han borrado {} registros duplicados.\n".format(final_data.place_id.duplicated().sum()))
# Se eliminan los duplicados
final_data = final_data.drop_duplicates(subset = ["place_id"])

El dataset contiene un total de 76177 registros.

Se han borrado 70965 registros duplicados.



En la ruta, poner el nombre de la persona que lo ha generado. Por ejemplo: "places_ARNAU.csv"

In [ ]:
# Se guarda el resultado en csv
path = "/content/drive/MyDrive/places_XXXXX.csv"
final_data.to_csv(path)